In [1]:
import argparse

from utils import (print_yaml_config,init_or_resume_from, read_yaml)
from model_training.training_utils import load_for_inference
from constants import TOKENIZER_SEPECIAL_TOKENS
from training_datasets.dataset_utils import load_rm_dataset

def main(conf):
    print(f"\n{'==='*10} Following are the configuration for training{'==='*10}")
    print_yaml_config(conf)
    device_map = {"":1}
    special_tokens = TOKENIZER_SEPECIAL_TOKENS["llama"]
    model, tokenizer = load_for_inference(device_map,conf,special_tokens,conf.model_name,True)
    return model, tokenizer

config = {}
conf = read_yaml('./config.yaml')
config.update(conf["default"])
config.update(conf["eval_ranking_rm"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "eval_ranking_rm"

# Create a Namespace object for config
config_ranking = argparse.Namespace(**config)

config = {}
config.update(conf["default"])
config.update(conf["eval_abs_rm"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "eval_abs_rm"

# Create a Namespace object for config
config_abs = argparse.Namespace(**config)

train_e , eval_abs = load_rm_dataset(config_abs)

train_er , eval_ranking = load_rm_dataset(config_ranking)


/home/alikhan/miniconda3/envs/torch_p310_setup/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


===loading the oasst_export_abs dataset===

============================== total 735 has None type quality label
============================== total 38 has None type quality label
OASST HF dataset: len(train)=38891, len(val)=2035
Size of oasst_export_abs training data: 38891
Size of oasst_export_abs validation data: 100
============================== Total training dataset size is 38891...
	============================== Validation size for oasst_export_abs dataset size is 100...
===loading the oasst_export dataset===

OASST HF dataset: len(train)=14244, len(val)=743
Size of oasst_export training data: 14244
Size of oasst_export validation data: 743
============================== Total training dataset size is 14244...
	============================== Validation size for oasst_export dataset size is 743...


In [2]:
# s = []

# for e in train_e:
#     prefix, reply, score = e
#     s.append(score)

# for e in eval_abs["oasst_export_abs"]:
#     prefix, reply, score = e
#     s.append(score)

# import matplotlib.pyplot as plt


# # Plotting the histogram
# plt.hist(s, bins=10, edgecolor="k", alpha=0.7)
# plt.title("Distribution of Float Numbers")
# plt.xlabel("Value")
# plt.ylabel("Frequency")
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.show()

In [2]:
# config_ranking.adapter_name = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
# config_ranking.init_from_adapter = "LLama-2-7b_ours_oasst_sft_reward_ranking_bs64"
# init_or_resume_from(config_ranking)
# config_ranking.model_name = config_ranking.merged_adapter_path if config_ranking.merged_adapter_path else config_ranking.base_model_name
# ranking_reward_model, ranking_reward_tokenizer = main(config_ranking)

# print('yay')

# config_abs.adapter_name = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
# config_abs.init_from_adapter = "LLama-2-7b_ours_oasst_sft_reward_abs_bs64"
# init_or_resume_from(config_abs)
# config_abs.model_name = config_abs.merged_adapter_path if config_abs.merged_adapter_path else config_abs.base_model_name
# abs_reward_model, abs_reward_tokenizer = main(config_abs)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_reward_ranking_bs64/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: null
adam_beta2: 0.999
adam_epsilon: null
adapter_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adapter_number: final_checkpoint
adpater_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adpater_path: output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint
base_model_name: meta-llama/Llama-2-7b-hf
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  oasst_export:
    lang: en,es,de,fr
    val_split: 0.05
debug: false
debug_set: 100
dtype: bf16
early_stopping: false
ev

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.08s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==============================Now Loading Peft model output/LLama-2-7b_ours_oasst_sft_reward_ranking_bs64/final_checkpoint.
tokenizer size 32003
Resizing embeddings to 32016
yay
============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_reward_abs_bs64/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: null
adam_beta2: 0.999
adam_epsilon: null
adapter_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adapter_number: final_checkpoint
adpater_name: LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default
adpater_path: output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint
base_model_name: meta-llama/Llama-2-7b-hf
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  syst

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.48s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


==============================Now Loading Peft model output/LLama-2-7b_ours_oasst_sft_reward_abs_bs64/final_checkpoint.
tokenizer size 32003
Resizing embeddings to 32016


In [2]:
import transformers
from transformers import BitsAndBytesConfig
from peft import PeftModel
from constants import DTYPES, CACHE_DIR
import torch


dtype = DTYPES.get(config_abs.dtype, torch.float32)
bnb_config =  BitsAndBytesConfig(
    load_in_4bit=config_abs.int4_training,
    load_in_8bit=config_abs.int8_training,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float32,
)

print(dtype)
print(bnb_config)

model_args = {
    "torch_dtype": bnb_config.bnb_4bit_compute_dtype,
    "quantization_config": bnb_config,
    "cache_dir": CACHE_DIR,
    "device_map": {"":1}
}
name = "output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged"
reward_base_model = transformers.AutoModelForSequenceClassification.from_pretrained(name, num_labels=1,**model_args)
reward_base_model = PeftModel.from_pretrained(reward_base_model,'output/LLama-2-7b_ours_oasst_sft_reward_ranking_bs64/final_checkpoint')
reward_base_model.load_adapter('output/LLama-2-7b_ours_oasst_sft_reward_abs_sigmoid_bs64/final_checkpoint',adapter_name="abs")


torch.bfloat16
BitsAndBytesConfig {
  "bnb_4bit_quant_type": "nf4",
  "load_in_4bit": true
}



Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.65s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/merged and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.weight', 'base_model.model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.rotary_emb.inv_freq', 'base_model.model.model.layer

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(name)

In [4]:
from training_datasets.collators import AbsoluteScoreDataCollator, RankingDataCollator


abs_collate_fn = AbsoluteScoreDataCollator(
    tokenizer,
    max_length=config_abs.collator["max_length"],
    pad_to_multiple_of=16,
)

ranking_collate_fn = RankingDataCollator(
    tokenizer,
    max_length=config_ranking.collator["max_length"],
    pad_to_multiple_of=16,
    max_replies=config_ranking.max_replies
)

# eval_abs,eval_ranking

In [15]:
# inputs, cu_lens = ranking_collate_fn([eval_ranking["oasst_export"][90]])
# inputs.input_ids.shape

# inputs = abs_collate_fn([eval_abs["oasst_export_abs"][90]])



/home/alikhan/miniconda3/envs/torch_p310_setup/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2411: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [23]:
idx = 99

# for idx in range(len(eval_abs["oasst_export_abs"])):
data = eval_abs["oasst_export_abs"][idx]
inputs = abs_collate_fn([data])
labels = inputs.pop("labels")
print(f'label:{labels}')

import torch
sigmoid = torch.nn.Sigmoid()

# data = [["Hi how are you?"],["I am good you piece of shit","Giberish, Giberish saying Giberish, tell Giberish is not Giberish. Why are Giberish you Giberish","I am good","I am good, how are you doing? Please tell me how can I help you? "]] 
# data = eval_ranking["oasst_export"][idx]
# inputs, _ = ranking_collate_fn([data])
print('***'*10)
for d in tokenizer.batch_decode(inputs.input_ids):
        print(f'{d}')


reward_base_model.set_adapter("abs")        
logits = reward_base_model(input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        use_cache=False,
        ).logits
print(sigmoid(logits),logits)
print('==='*10)

reward_base_model.set_adapter("default")        
logits = reward_base_model(input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        use_cache=False,
        ).logits
print(sigmoid(logits),logits)
print('==='*10)


# print(abs_reward_tokenizer.batch_decode(inputs["input_ids"]))        


label:tensor([0.5000], dtype=torch.float64)
******************************
<s><|prompter|> Briefly and concisely explain what placenta previa is and include one priority action to take as a nurse when managing a patient with this disorder</s><|assistant|> Placenta previa is a pregnancy complication in which the placenta is located low in the uterus and partially or completely covers the cervix. This can cause bleeding, particularly during the later stages of pregnancy or during labor.

One priority action for a nurse managing a patient with placenta previa is to closely monitor the patient's bleeding and vital signs, as well as the fetal heart rate. If the bleeding becomes severe, it may be necessary to prepare the patient for an emergency cesarean section to protect the health of the mother and baby. The nurse should also provide emotional support and education to the patient and her family about the condition and the need for close monitoring and medical intervention.</s><|prompter|>

In [23]:
tokenizer.decode(inputs.input_ids[1])

'<s><|prompter|> Hi how are you?</s><s><|assistant|> I am good</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>'

## SFT


In [2]:
import argparse

from utils import (print_yaml_config,init_or_resume_from, read_yaml)
from model_training.training_utils import load_for_inference
from constants import TOKENIZER_SEPECIAL_TOKENS
from training_datasets.dataset_utils import load_sft_dataset, load_rm_dataset
from training_datasets.collators import DialogueDataCollator




def main(conf):
    print(f"\n{'==='*10} Following are the configuration for training{'==='*10}")
    print_yaml_config(conf)
    device_map = {"":1}
    special_tokens = TOKENIZER_SEPECIAL_TOKENS["llama"]
    model, tokenizer = load_for_inference(device_map,conf,special_tokens,conf.model_name,False)
    return model, tokenizer

config = {}
conf = read_yaml('./config.yaml')
config.update(conf["default"])
config.update(conf["sft_eval"])
config["name_suffix"] = ""
config["debug"] = False
config["subset"] = "sft_eval"

# Create a Namespace object for config
config_ns = argparse.Namespace(**config)

init_or_resume_from(config_ns)

/home/alikhan/miniconda3/envs/torch_p310_setup/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


============================== Initialize from adapter output/LLama-2-7b_pre_sft_warm_20/final_checkpoint


In [3]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep2_qlora_sft_model, ep2_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: null
adam_beta2: 0.999
adam_epsilon: null
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:44<00:00, 22.22s/it]


==============================Now Loading Peft model output/LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default/final_checkpoint.
tokenizer size 32003
Resizing embeddings to 32016


In [5]:

ep2_qlora_sft_model.load_adapter('output/LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default/final_checkpoint',adapter_name='a')


_IncompatibleKeys(missing_keys=['base_model.model.model.embed_tokens.original_module.weight', 'base_model.model.model.embed_tokens.modules_to_save.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_mo

In [4]:
config_ns.model_name = "meta-llama/Llama-2-7b-hf"
config_ns.init_from_adapter = "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default"
config_ns.adapter_number= "final_checkpoint"
init_or_resume_from(config_ns)
ep3_qlora_sft_model, ep3_qlora_sft_tokenizer = main(config_ns)

============================== Initialize from adapter output/LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default/final_checkpoint

============================== Following are the configuration for training==============================
!!python/object:argparse.Namespace
adam_beta1: 0.9
adam_beta2: 0.999
adam_epsilon: 1e-12
adapter_number: final_checkpoint
adpater_name: null
checkpoint_name: null
checkpoint_number: final_checkpoint
collator:
  label_masking: true
  max_length: 2048
  random_offset_probability: 0.5
  samples_mixing: false
  system_prefix: null
  use_system_prefix: false
dataset:
  alpaca:
    max_val_set: 200
    val_split: 0.05
  dolly:
    max_val_set: 300
    val_split: 0.05
  math_instruction:
    max_val_set: 200
    val_split: 0.05
  oasst_export:
    lang: en,bg,ca,cs,da,de,en,es,fr,hr,hu,it,nl,pl,pt,ro,ru,sl,sr,sv,uk
    max_val_set: null
    val_split: 0.05
  vicuna:
    max_val_set: 800
    val_split: 0.05
  webgpt:
    max_val_set: 1000
    val_s

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.35s/it]


tokenizer size 32003
Resizing embeddings to 32016


In [11]:
m_t = {
       # "LLama-2-7b_full_sft_4bit_bs_64":[full_sft_model,full_sft_tokenizer,0.8,0.9,50],
       "LLama-2-7b_ours_oasst_sft_ep_2_lr_1e4_cosine_bs64_qlora_default":[ep2_qlora_sft_model, ep2_qlora_sft_tokenizer,0.8,0.9,40],
       # "LLama-2-7b_ours_oasst_sft_only_bs64_qlora_default": [only_oasst_model, only_oasst_tokenizer,0.85,0.9,40],
       "LLama-2-7b_ours_oasst_sft_ep_3_lr_1e4_cosine_bs64_qlora_default":[ep3_qlora_sft_model, ep3_qlora_sft_tokenizer,0.8,0.9,0]
}

In [7]:
def get_pred(m_t,input):
    for key,value in m_t.items():
        model,tokenizer,temp,top_p,top_k = value
        tokens = tokenizer(input,return_tensors='pt')
        output = model.generate(**tokens,
                        max_new_tokens=512,
                        do_sample=True,
                        top_p=top_p,
                        top_k=top_k,
                        repetition_penalty=1.2,
                        temperature=temp,
                    )
        print(f"\n\nOutput for model {key}:\n{'-'*100}")
        print(tokenizer.decode(output[0], skip_special_tokens=False))




In [21]:

input = """<|prompter|>Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|>"""
get_pred(m_t,input)




Output for model LLama-2-7b_full_sft_4bit_bs_64:
----------------------------------------------------------------------------------------------------
<s><|prompter|> Considering the advancements in quantum computing and its potential implications for classical encryption methods, explain how current cybersecurity measures might need to adapt, and what challenges or benefits this new computational paradigm could present to the fields of data privacy and digital communication.</s><|assistant|> With the advent of quantum computers and their ability to break certain types of cryptographic algorithms used in traditional security measures such as RSA and elliptic curve, it is important that existing systems are updated with newer, more secure ones which can be protected against these attacks.
The main challenge faced by organizations will likely include creating a balance between providing adequate levels of protection while also keeping up with rapidly changing technologies.  Additionally